In [3]:
### !pip install scikit-surprise
# SVD recommend system SGD description: 
# http://surprise.readthedocs.io/en/stable/matrix_factorization.html

In [17]:
import numpy as np 

def load(file='input.txt'):
    fin = open(file)
    k, U, M, D, T = map(int, fin.readline().split())
    train, test = [], []
    i = 0
    while i < D + T:
        if i < D:
            line = fin.readline().split()
            train.append((int(line[0]), int(line[1]), float(line[2])))
        else:
            test.append(tuple(map(int, fin.readline().split())))
        i += 1  
    fin.close()
    return train, test, sum([o[2] for o in train]) / D, U, M, k

def fit_svd(train, rating_mean, n_users, n_movies):

    bu = np.zeros(n_users)
    bm = np.zeros(n_movies)
    qm = np.random.normal(0, 0.05, (n_movies, 10))
    pu = np.random.normal(0, 0.05, (n_users, 10))

    epochs = 10
    lam = 0.02
    gam = 0.015

    # SGD 
    for _ in range(epochs):
        for u, m, r in train:
            r_hat = rating_mean + bu[u] + bm[m] + np.dot(pu[u], qm[m])
            err = r - r_hat

            bu[u] = bu[u] + gam * (err - lam * bu[u])
            bm[m] = bm[m] + gam * (err - lam * bm[m])

            pu[u] = pu[u] + gam * (err * qm[m] - lam * pu[u])
            qm[m] = qm[m] + gam * (err * pu[u] - lam * qm[m])
    
    svd_params = (rating_mean, bu, bm, pu, qm)
    return svd_params

def predict(test, svd_params, min_value, max_value):
    rating_mean, bu, bm, pu, qm = svd_params
    pred = []
    for u, m in test:
        r = rating_mean + bu[u] + bm[m] + np.dot(pu[u], qm[m])
        r = max(min_value, min(r, max_value))
        pred.append(r)
    return pred 

# pipeline 
train, test, rating_mean, n_users, n_movies, max_value = load()
svd_params = fit_svd(train, rating_mean, n_users, n_movies)
pred = predict(test, svd_params, 2, max_value)
print(*pred, sep='\n')

7.189234944829974
6.631265020596081
7.114199667781863
6.615160493599864
